In [22]:
import pandas as pd
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import HashingVectorizer,CountVectorizer,TfidfVectorizer  
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier

**Reboot: Box-Plots for Education competition**

**Data importation : training data**

In [23]:
train_data = pd.read_csv('../data/TrainingData.csv', index_col=[0])

In [24]:
train_data.head(5)

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [25]:
X = train_data.loc[:, 'Object_Description':]
X.head(5)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [26]:
y = train_data.loc[:, :'Operating_Status']
y.head(5)

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating


**Data importation : testing data**

In [27]:
X_test = pd.read_csv('../data/TestData.csv', index_col=[0])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
X_test.head(5)

,Object_Description,Program_Description,SubFund_Description,Job_Title_Description,Facility_or_Department,Sub_Object_Description,Location_Description,FTE,Function_Description,Position_Extra,Text_4,Total,Text_2,Text_3,Fund_Description,Text_1
180042,Student Meals/Room/Other,Basic Educational Services,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,NaN,NaN,General Fund,NaN
28872,Extra Duty/Signing Bonus Pay,Undistributed,NaN,CHEERLEADER DIR,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,NaN,NaN,General Fund,NaN
186915,Professional Salaries,Bilingual Education,NaN,T-EL 1ST BIL,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,NaN,NaN,General Fund,NaN
412396,Professional Salaries,Bilingual Education,NaN,T-EL 2ND BIL,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,NaN,NaN,General Fund,NaN
427740,Salaries for Support Personnel,Undistributed,NaN,CLERK III- SCH,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,NaN,NaN,General Fund,NaN


**Test data column ordering must be equal to Training data**

In [29]:
X_test1 = pd.DataFrame(X_test,columns=X.columns)

In [30]:
X_test1.head(5)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
180042,Student Meals/Room/Other,NaN,NaN,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,Basic Educational Services,General Fund,NaN
28872,Extra Duty/Signing Bonus Pay,NaN,NaN,CHEERLEADER DIR,NaN,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,Undistributed,General Fund,NaN
186915,Professional Salaries,NaN,NaN,T-EL 1ST BIL,NaN,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,Bilingual Education,General Fund,NaN
412396,Professional Salaries,NaN,NaN,T-EL 2ND BIL,NaN,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,Bilingual Education,General Fund,NaN
427740,Salaries for Support Personnel,NaN,NaN,CLERK III- SCH,NaN,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,Undistributed,General Fund,NaN


**Inspect Data Types**

In [31]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 16 columns):
Object_Description        375493 non-null object
Text_2                    88217 non-null object
SubFund_Description       306855 non-null object
Job_Title_Description     292743 non-null object
Text_3                    109152 non-null object
Text_4                    53746 non-null object
Sub_Object_Description    91603 non-null object
Location_Description      162054 non-null object
FTE                       126071 non-null float64
Function_Description      342195 non-null object
Facility_or_Department    53886 non-null object
Position_Extra            264764 non-null object
Total                     395722 non-null float64
Program_Description       304660 non-null object
Fund_Description          202877 non-null object
Text_1                    292285 non-null object
dtypes: float64(2), object(14)
memory usage: 51.9+ MB


**Seperate Numerical and Text Features**

In [32]:
num_cols = X.select_dtypes(include='number').columns.to_list()
text_cols = X.select_dtypes(exclude='number').columns.to_list()
text_cols

['Object_Description',
 'Text_2',
 'SubFund_Description',
 'Job_Title_Description',
 'Text_3',
 'Text_4',
 'Sub_Object_Description',
 'Location_Description',
 'Function_Description',
 'Facility_or_Department',
 'Position_Extra',
 'Program_Description',
 'Fund_Description',
 'Text_1']

In [33]:
X_test1.shape

(50064, 16)

**Create csv submission file**

In [34]:
def create_csv_submission_file(model, X_test, y_train):
    # Make prediction
    y_pred = model.predict_proba(X_test)
    
    # Make list of column names
    pc_pairs = zip(y_train.columns, model.named_steps.classifier.classes_)
    columns = []
    for parent, children in pc_pairs:
        for child in children:
            columns.append(f'{parent}__{child}')
            
    # Put predictions in DataFrame
    df = pd.DataFrame(np.concatenate(y_pred, axis=1),
                      index=X_test.index,
                      columns=columns)
    df = df[sorted(df.columns)]
    
    date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
    df.to_csv(f'../predictions/{date}submission.csv', index=True, header=True)

**Custom transformer to join all the text in the text columns**

In [35]:
class Concatenator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        results = []
        for row in X:
            results.append(' '.join([w for w in row if w]))
        return results

**Using DecisionTreeClassifier**

In [ ]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', HashingVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

model = Pipeline([
    ('ct', ct),
    ('classifier', DecisionTreeClassifier(max_depth=3))
])


In [ ]:
model.fit(X,y);

In [ ]:
create_csv_submission_file(model, X_test1, y)

In [ ]:
y_pred = model.predict_proba(X_test1)

In [ ]:
pd.DataFrame(np.concatenate(y_pred, axis=1),index = X_test1.index).head()

In [ ]:
pd.DataFrame(np.hstack(y_pred),index = X_test1.index).head()

In [ ]:
y.columns

In [ ]:
model.named_steps.classifier.classes_

In [ ]:
# Make list of column names
pc_pairs = zip(y.columns, model.named_steps.classifier.classes_)
columns = []
for parent, children in pc_pairs:
    for child in children:
        columns.append(f'{parent}__{child}')

In [ ]:
columns

In [ ]:
pd.DataFrame(np.concatenate(y_pred, axis=1),
                      index=X_test1.index,
                      columns=columns).head()

In [ ]:
pd.DataFrame(np.hstack(y_pred),
             index = X_test1.index,
             columns=columns).head()

**Using RandomForestClassifier**

In [ ]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', HashingVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

randommodel = Pipeline([
    ('ct', ct),
    ('classifier', RandomForestClassifier(n_jobs=10,n_estimators=100,verbose=1))
])

In [ ]:
randommodel.fit(X,y);

In [ ]:
create_csv_submission_file(randommodel, X_test1, y)

**Using ExtraTreesClassifier**

In [48]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', HashingVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

extramodel = Pipeline([
    ('ct', ct),
    ('classifier', ExtraTreesClassifier(n_jobs=10,n_estimators=100,verbose=1))
])

In [49]:
extramodel.fit(X,y);

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 412.3min
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed: 1405.9min finished


In [50]:
create_csv_submission_file(extramodel, X_test1, y)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  9.1min
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed: 27.5min finished


**Using RadiusNeighborsClassifier**

In [ ]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', TfidfVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

radius = Pipeline([
    ('ct', ct),
    ('classifier', RadiusNeighborsClassifier(n_jobs=10))
])

In [ ]:
radius.fit(X,y);

In [ ]:
create_csv_submission_file(radius, X_test1, y)

**Using KNeighborsClassifier**

In [ ]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', TfidfVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

neighbor = Pipeline([
    ('ct', ct),
    ('classifier', KNeighborsClassifier(n_jobs=10))
])

In [ ]:
neighbor.fit(X,y);

In [ ]:
create_csv_submission_file(neighbor, X_test1, y)

**MultiOutputClassifier and XGBClassifier**

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:


num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', TfidfVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

clf = MultiOutputClassifier(XGBClassifier())

multiOutput = Pipeline([
    ('ct', ct),
    ('classifier', clf)
])


In [60]:
multiOutput.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ct',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num_pipe',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                                    

In [61]:
preds = multiOutput.predict(X_test)

In [112]:
multiOutput.named_steps.classifier.estimators_[0].classes_

array(['Aides Compensation', 'Career & Academic Counseling',
       'Communications', 'Curriculum Development',
       'Data Processing & Information Services',
       'Development & Fundraising', 'Enrichment',
       'Extended Time & Tutoring', 'Facilities & Maintenance',
       'Facilities Planning',
       'Finance, Budget, Purchasing & Distribution', 'Food Services',
       'Governance', 'Human Resources',
       'Instructional Materials & Supplies', 'Insurance', 'Legal',
       'Library & Media', 'NO_LABEL', 'Other Compensation',
       'Other Non-Compensation', 'Parent & Community Relations',
       'Physical Health & Services', 'Professional Development',
       'Recruitment', 'Research & Accountability',
       'School Administration', 'School Supervision', 'Security & Safety',
       'Social & Emotional',
       'Special Population Program Management & Support',
       'Student Assignment', 'Student Transportation',
       'Substitute Compensation', 'Teacher Compensation',
   

In [107]:
len(multiOutput.named_steps.classifier.estimators_)

9

In [110]:
length = len(multiOutput.named_steps.classifier.estimators_)
results = [multiOutput.named_steps.classifier.estimators_[n].classes_ for n in range(length)]


In [111]:
results

[array(['Aides Compensation', 'Career & Academic Counseling',
        'Communications', 'Curriculum Development',
        'Data Processing & Information Services',
        'Development & Fundraising', 'Enrichment',
        'Extended Time & Tutoring', 'Facilities & Maintenance',
        'Facilities Planning',
        'Finance, Budget, Purchasing & Distribution', 'Food Services',
        'Governance', 'Human Resources',
        'Instructional Materials & Supplies', 'Insurance', 'Legal',
        'Library & Media', 'NO_LABEL', 'Other Compensation',
        'Other Non-Compensation', 'Parent & Community Relations',
        'Physical Health & Services', 'Professional Development',
        'Recruitment', 'Research & Accountability',
        'School Administration', 'School Supervision', 'Security & Safety',
        'Social & Emotional',
        'Special Population Program Management & Support',
        'Student Assignment', 'Student Transportation',
        'Substitute Compensation', 'Teacher 

In [114]:
create_csv_submission_xg_file(multiOutput, X_test1, y)

In [113]:
def create_csv_submission_xg_file(model, X_test, y_train):
    # Make prediction
    y_pred = model.predict_proba(X_test)
    
    length = len(model.named_steps.classifier.estimators_)
    results = [model.named_steps.classifier.estimators_[n].classes_ for n in range(length)]
    # Make list of column names
    pc_pairs = zip(y_train.columns, results)
    columns = []
    for parent, children in pc_pairs:
        for child in children:
            columns.append(f'{parent}__{child}')
            
    # Put predictions in DataFrame
    df = pd.DataFrame(np.concatenate(y_pred, axis=1),
                      index=X_test.index,
                      columns=columns)
    df = df[sorted(df.columns)]
    
    date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
    df.to_csv(f'../predictions/{date}submission.csv', index=True, header=True)

In [97]:
extramodel.named_steps.classifier.classes_

[array(['Aides Compensation', 'Career & Academic Counseling',
        'Communications', 'Curriculum Development',
        'Data Processing & Information Services',
        'Development & Fundraising', 'Enrichment',
        'Extended Time & Tutoring', 'Facilities & Maintenance',
        'Facilities Planning',
        'Finance, Budget, Purchasing & Distribution', 'Food Services',
        'Governance', 'Human Resources',
        'Instructional Materials & Supplies', 'Insurance', 'Legal',
        'Library & Media', 'NO_LABEL', 'Other Compensation',
        'Other Non-Compensation', 'Parent & Community Relations',
        'Physical Health & Services', 'Professional Development',
        'Recruitment', 'Research & Accountability',
        'School Administration', 'School Supervision', 'Security & Safety',
        'Social & Emotional',
        'Special Population Program Management & Support',
        'Student Assignment', 'Student Transportation',
        'Substitute Compensation', 'Teacher 